# Guideline to prepare files for methylation calling with modkit

For methylation calling we choosed to use reads basecall with Dorado, as well as their respective assemblies construct from Dorado basecalling. As dorado was much more quicker to perform than Guppy and output similar basecalling accuracy and assemblies quality to Guppy.

## I. Dorado trimming

First, we need to trim our adapters from the ubam files  produced by Dorado basecalling, normally it is done by default by Dorado, but as a precausion was run again

Note: ubam file are the official format of Dorado basecalling to stock methylation data (it is based on bam file but unaligned, and contain an MM and ML tag prone for methylation storage).It contain the same information as a fastq, with the identifier of the read, the corresponding sequence, and the quality.

## II. Dorado aligner

Once adapters are removed from the reads, we need to aligned our ubam to our reference genome (the one obtain from HyPo polishing)


## III. Samtools sort + index

After aligning or ubam to their reference genome, we need to first sort, then index the files to be able to call the methylation with modkit

## Code to perform trimming, aligning, and sorting + indexing

In [ ]:
#!/bin/bash

# Find all files in the current directory with the specified extension
files=$(find . -type f -name "*_dorado_modbasecalling.bam")

# Iterate over each file found
for file in $files
do
    # Extract the filename without extension
    filename=$(basename "$file" "_dorado_modbasecalling.bam")

    # Execute the dorado trim command for each file
    dorado trim "$file" > "${filename}_trimmed.bam" -t 50

    # Execute the alignment command for each trimmed BAM file
    dorado aligner /bigvol/omion/HyPo/hypo_"$filename"_Dorado_modbasecalling.fasta ./"$filename"_trimmed.bam --bandwidth "500,20000" -t 50 > "aligned_trimmed_$filename.bam"

    # Sort the aligned BAM file
    samtools sort "aligned_trimmed_$filename.bam" -O BAM -o "aligned_sort_$filename.bam" -@ 50

    # Index the sorted BAM file
    samtools index -@ 50 "aligned_sort_$filename.bam"

done

# Modkit

Now we can process the resulting files with Modkit which will output .tsv or .bed files containing methylation information:,reference position of the methylation at a CpG site in the genome, type of methylation, probability of methylation...

## I. Modkit Pileup
Allow to obtain the probability of methylation for at each CpG site. No filtering was performed (no methylation threshold), in order to keep all possible methylated sites. + and - strand were combine using --combine-strand flag, and motif bed files containing the CpG position in their respective assemblies were created, and use in modkit pileup.

## Creation of the motif bed files for modkit pileup

In [ ]:
#!/bin/bash

# Directory containing the files
input_dir="/bigvol/omion/07-Filtered_Assemblies"

# Output directory
output_dir="/bigvol/omion/07-Filtered_Assemblies"

# Loop through all files matching the pattern
for file in "$input_dir"/filtered_hypo_Gd*_Dorado_modbasecalling.fasta; do
    # Extract the filename without path
    filename=$(basename "$file")

    # Generate output filename
    output_file="${output_dir}/${filename%.fasta}_cg_modifs.bed"

    # Execute the command on the file
    /bigvol/omion/Software/dist/modkit motif-bed "$file" CG 0 > "$output_file"
done


## Creation of the .bed files containing methylation data (modkit pileup)

In [ ]:
#!/bin/bash

# Define directories
base_dir="/bigvol/omion"
input_dir="$base_dir/11-Methylation/02-samtools_sort_index"
ref_dir="$base_dir/07-Filtered_Assemblies"
bed_dir="$base_dir/11-Methylation/03-modkit-motif_bed"
output_dir="$base_dir/11-Methylation/04-Modkit_pileup"

# Create output directory
mkdir -p "$output_dir"

# Process each BAM file
for bam_file in "$input_dir"/aligned_sort_Gd*.bam; do
    # Extract sample name
    sample_name=$(basename "$bam_file" | sed 's/aligned_sort_\(Gd[0-9]*\)\.bam/\1/')

    # Define file paths
    output_file="$output_dir/pileup_${sample_name}.bed"
    ref_file="$ref_dir/filtered_hypo_${sample_name}_Dorado_modbasecalling.fasta"
    include_bed="$bed_dir/filtered_hypo_${sample_name}_Dorado_modbasecalling_cg_modifs.bed"

    # Run modkit pileup command
    /bigvol/omion/Software/dist/modkit pileup "$bam_file" "$output_file" \
        --combine-strands --motif CG 0 \
        --ref "$ref_file" \
        --include-bed "$include_bed" \
        --no-filtering -t 50

    echo "Processed $sample_name"
done

echo "All files processed."

## Calculate Genome-wide 5mCpG and 5hmCpG %

We calculated the mean % of 5mC and 5hmC at CpG sites using the following command lines:

The calculation is based on the number of reads that have a detected methylation mark at a given genomic position.
Therefore, the calculation is for each position:
Number of aligned CpG with 5mC (or 5hmC) / Number of total CpG

In [ ]:
## Code to obtain global number of methylated sites (comprise both 5mC and 5hmC as the model is probabilistic)
awk '($4=="m") && ($11>0.0)' ${pileup_bed} | wc -l ## specify file name

## Code to obtain global methylation level (5mC, 5hmC)
awk '$4=="m" {can+=$13; mod+=$12; oth+=$14; valid+=$10} \
  END{print (can/valid) " CpG canonical\n" (mod/valid) " 5mCpG modified\n" (oth/valid) " 5hmCpG modified"}' ${pileup_bed} ## specify file name

Data were stored in the file: Genome_wide_methylation.csv

##Rscript to plot Genome-wide methylation accross isolates

In [ ]:
setwd("C:/Users/ocean/Downloads")

library(ggplot2)
library(dplyr)
library(gridExtra)
library(ggtext)

# Read the data
data <- read.delim("Genome_wide_methylation(1).csv", header = TRUE)

# Assign colors based on category in column 2
data$color <- case_when(
  data[[2]] == 1 ~ "#FF5733",
  data[[2]] == 2 ~ "#3377FF",
  data[[2]] == "Outgroup" ~ "#9ACD32",
  TRUE ~ "black"  # Default color for any other categories
)

# Function to order data by color and a specific column in decreasing order
order_data <- function(data, column_index) {
  data %>%
    mutate(color = factor(color, levels = c("#FF5733", "#3377FF", "#9ACD32"))) %>%
    arrange(color, desc(data[[column_index]]))
}

# Order the data for each plot
data1 <- order_data(data, 3)
data2 <- order_data(data, 4)
data3 <- order_data(data, 5)

# Create a factor for the first column based on the new order for each plot
data1[[1]] <- factor(data1[[1]], levels = data1[[1]])
data2[[1]] <- factor(data2[[1]], levels = data2[[1]])
data3[[1]] <- factor(data3[[1]], levels = data3[[1]])

# Create the three plots
plot1 <- ggplot(data1, aes(x = data1[[1]], y = data1[[3]], fill = color)) +
  geom_col() +
  scale_fill_identity() +
  labs(x = colnames(data1)[1], y = "Nb of methylated sites",
       title = "Number of methylated sites per isolates") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

plot2 <- ggplot(data2, aes(x = data2[[1]], y = data2[[4]], fill = color)) +
  geom_col() +
  scale_fill_identity() +
  labs(x = colnames(data2)[1], y = "5mCpG %",
       title = "Genome wide 5mCpG % per isolates") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

plot3 <- ggplot(data3, aes(x = data3[[1]], y = data3[[5]], fill = color)) +
  geom_col() +
  scale_fill_identity() +
  labs(x = colnames(data3)[1], y = "5hmCpG %",
       title = "Genome wide 5hmCpG % per isolates") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        plot.title = element_markdown(hjust = 0.5))

# Arrange the plots in a grid
grid.arrange(plot1, plot2, plot3, ncol = 2)


Note: To have all the genomic position and not only the CpG, it is possible to perform a bedtools complement.

The goals is to have the complement position of the pileup output. Indeed pileup will only give you the position of the genome where methylation is present.
You will need to create genome file before with the first column being the contig name and the second the lengh of the contig.

Then this complement file will be merge using a cat command with the pileup to have all the genomic position (those file can allow the profiling of methylation along the genome).

In [ ]:
#!/bin/bash

# Directory containing the input pileup files
input_dir="./"

# Directory containing the contig length files
contig_length_dir="/bigvol/omion/07-Filtered_Assemblies/contig_lengh/"

# Output directory (same as input in this case)
output_dir="./"

# Loop through all pileup files matching the pattern Gd*
for pileup_file in ${input_dir}pileup_Gd*.bed; do
    # Extract the sample name
    sample_name=$(basename "$pileup_file" | sed 's/pileup_\(Gd[0-9]*\)\.bed/\1/')

    # Construct the contig length file name
    contig_length_file="${contig_length_dir}${sample_name}_contig_length.txt"

    # Construct the output file name
    output_file="${output_dir}pileup_complement_${sample_name}.bed"

    # Run the bedtools complement command
    bedtools complement -i "$pileup_file" -g "$contig_length_file" | \
    awk -v OFS='\t' '{for (i=$2; i<$3; i++) print $1, i, i+1, ".", 0, 0.0}' > "$output_file"

    echo "Processed $sample_name"
done

echo "All files processed."


## Merge complement + pileup

In [ ]:
#!/bin/bash

# Iterate over each pair of files
for file in ./pileup_complement_Gd*.bed; do
    # Extracting the ID from the filename
    id="${file#./pileup_complement_Gd}"
    id="${id%.bed}"

    # Constructing the corresponding tmp_pileup filename
    tmp_file="./awk_pileup_Gd${id}.bed"

    # Check if the tmp_pileup file exists
    if [ -e "$tmp_file" ]; then
        # Merge and sort the files
        cat "$file" "$tmp_file" | bedtools sort > "pileup_complemented_Gd${id}.bed"

        # Print a message indicating the completion
        echo "Merged and sorted files for $file and $tmp_file"
    else
        echo "Error: Corresponding tmp_pileup file not found for $file"
    fi
done


## Calculate contig-wide 5mCpG and 5hmCpG %

Then we choose to also see if some contigs from our assemblies display more or less methylation

In [ ]:
#!/bin/bash

awk '
    BEGIN {
        print "Contig\t5mCpG\t5hmCpG"
    }
    $4=="m" {
        contig=$1
        mod[contig]+=$12
        oth[contig]+=$14
        valid[contig]+=$10
    }
    END {
        for (contig in valid) {
            printf "%s\t%.6f\t%.6f\n", contig, mod[contig]/valid[contig], oth[contig]/valid[contig]
        }
    }
' ${pileup_bed} | (sed -u 1q; sort)

Data were stored in the file: contig_wide_methylation.csv

##Rscript for contig wide methylation

In [ ]:
setwd("C:/Users/ocean/Desktop")

# Load necessary libraries
library(ggplot2)
library(dplyr)
library(gridExtra)

# Read the CSV file
data <- read.delim("contig_wide_methylation.csv", header = TRUE, sep=",")

# Rename columns for clarity (adjust these names as needed)
colnames(data)[c(2,3,4,5)] <- c("Legend", "name", "category4", "value")

# Function to filter unique contig names and reorder by value
filter_unique_contigs_and_reorder <- function(df) {
  df <- df %>%
    group_by(name) %>%
    filter(row_number() == 1) %>%
    ungroup() %>%
    arrange(desc(value))
  return(df)
}

# Define colors for each category in Legend
colors <- c("1" = "#FF5733", "2" = "#3377FF", "Outgroup" = "#9ACD32")

# Create separate plots for 'h' and 'm', with unique contig names and ordered by value
plot_h <- data %>%
  filter(category4 == "h") %>%
  filter_unique_contigs_and_reorder() %>%
  ggplot(aes(x = reorder(name, -value), y = value, fill = Legend)) +  # Use reorder for ordering
  geom_bar(stat = "identity", position = position_dodge(width = 0.9), color = "black") +
  facet_wrap(~ Legend, ncol = 1, scales = "free_x") +
  scale_x_discrete(expand = c(0, 0)) +
  scale_y_continuous(expand = c(0, 0), limits = NULL, name = "5hmCpG %") +  # Y-axis label for the first plot
  scale_fill_manual(values = colors) +  # Use manual scale for fill colors
  theme_minimal() +
  theme(
    axis.text.x = element_blank(),
    axis.ticks.x = element_blank(),
    panel.grid.major.x = element_blank(),
    panel.grid.minor.x = element_blank(),
    panel.spacing = unit(0.1, "lines")
  ) +
  labs(title = expression(paste("Mean 5hmCpG % in ", italic("P. destructans"), " and outgroup contigs")), x = NULL, y = NULL)

plot_m <- data %>%
  filter(category4 == "m") %>%
  filter_unique_contigs_and_reorder() %>%
  ggplot(aes(x = reorder(name, -value), y = value, fill = Legend)) +  # Use reorder for ordering
  geom_bar(stat = "identity", position = position_dodge(width = 0.9), color = "black") +
  facet_wrap(~ Legend, ncol = 1, scales = "free_x") +
  scale_x_discrete(expand = c(0, 0)) +
  scale_y_continuous(expand = c(0, 0), limits = NULL, name = "5mCpG %") +  # Y-axis label for the second plot
  scale_fill_manual(values = colors) +  # Use manual scale for fill colors
  theme_minimal() +
  theme(
    axis.text.x = element_blank(),
    axis.ticks.x = element_blank(),
    panel.grid.major.x = element_blank(),
    panel.grid.minor.x = element_blank(),
    panel.spacing = unit(0.1, "lines")
  ) +
  labs(title = expression(paste("Mean 5mCpG % in ", italic("P. destructans"), " and outgroup contigs")), x = NULL, y = NULL)

# Arrange and display plots in two rows
grid.arrange(plot_h, plot_m, nrow = 2)
